In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import scanpy as sc
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso
from sklearn.metrics import accuracy_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [2]:
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pytorch_lightning as pl

/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
# make GPU visible
os.environ["CUDA_VISIBLE_DEVICES"]="3"
torch.cuda.is_available()

True

In [4]:
import scgpt as scg
import sys
from pathlib import Path

In [5]:
import random

def set_random_seed(seed_value=42):
    """Set random seed for reproducibility."""
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if using CUDA
    np.random.seed(seed_value)
    random.seed(seed_value)

# Call the function before creating your Lightning module, data module, or initializing your data loaders
set_random_seed()

# data: cd8 t cell (rarest cell population) pheno shift - 1000 patients

#### data is created in scSet/datasets/create_sim_data_scvi_pheno.ipynb (or a corresponding script)

In [6]:
h5ad_loc="/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_adata.h5ad"

In [7]:
# read in data from cache
adata = sc.read_h5ad(h5ad_loc)

/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
adata

AnnData object with n_obs × n_vars = 445645 × 23776
    obs: 'orig_cellbarcode', 'patient_orig', 'celltype', 'Tcellsubtype', 'group', 'patient', 'n_genes'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'celltype_colors', 'group_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts', 'lognorm'
    obsp: 'connectivities', 'distances'

In [9]:
adata.obs.rename(columns={'celltype':'cell_type'}, inplace=True)

### embed cells using scGPT

In [10]:
model_dir = Path("/data/rna_rep_learning/scGPT/scGPT_human")

In [11]:
adata = scg.tasks.embed_data(
    adata,
    model_dir,
    gene_col="index",
    batch_size=64,
    return_new_adata=False,
)

scGPT - INFO - match 21507/23776 genes in vocabulary of size 60697.


Embedding cells: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 6964/6964 [08:43<00:00, 13.29it/s]
/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/scgpt/tasks/cell_emb.py:279: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings
/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [12]:
#save this new adata
adata.write("/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_adata_scGPT.h5ad")

In [ ]:
adata = sc.read_h5ad("/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_adata_scGPT.h5ad")

## define sample pool, target, X, splits for this task

### define sample_pool for this target

In [13]:
sample_col = 'patient'
target_col = 'group'

In [14]:
adata.obs[[sample_col, target_col]].drop_duplicates()[target_col].value_counts()

group
perturbed      500
unperturbed    500
Name: count, dtype: int64

In [15]:
sample_pool = adata.obs[sample_col].drop_duplicates()
print(len(sample_pool))

1000


In [16]:
sample_pool

0       sim_pt1
0       sim_pt2
0       sim_pt3
0       sim_pt4
0       sim_pt5
        ...    
0     sim_pt996
0     sim_pt997
0     sim_pt998
0     sim_pt999
0    sim_pt1000
Name: patient, Length: 1000, dtype: category
Categories (1000, object): ['sim_pt1', 'sim_pt2', 'sim_pt3', 'sim_pt4', ..., 'sim_pt997', 'sim_pt998', 'sim_pt999', 'sim_pt1000']

### define target, and limit/order by sample_pool

In [17]:
## Define targets
targets = adata.obs.loc[:, [sample_col, target_col]].drop_duplicates().set_index(sample_col).loc[sample_pool,target_col]
labels, targets_ind = np.unique(targets, return_inverse=True)
targets = pd.Series(targets_ind, index=targets.index)

In [18]:
labels

array(['perturbed', 'unperturbed'], dtype=object)

In [19]:
targets.value_counts()/len(targets)

1    0.5
0    0.5
Name: count, dtype: float64

### 10 fold nested cross validation (cv outer loop)

In [21]:
skf = StratifiedKFold(n_splits=10)
splits = list(skf.split(np.zeros(len(targets)), targets))

In [22]:
for (train_index, test_index) in splits:
    #print("datasets reserved for test fold: ", list(adata.obs.set_index(sample_col).loc[sample_pool[test_index]].dataset.drop_duplicates()))
    print("n samples in train: ", len(train_index))
    print("n samples in test: ", len(test_index))
    print("\n")

n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100


n samples in train:  900
n samples in test:  100




## create pseudobulk embeddings to use downstream

In [23]:
pbulk_embs = {}

#### scGPT

In [24]:
emb_name = 'X_scGPT'

## Input
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1031071/1669887707.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### scGPT -> 40 PCs

In [25]:
emb_name = 'X_scGPT'

pca = sklearn.decomposition.PCA(n_components=40)
scgpt_input_pca = pca.fit_transform(adata.obsm[emb_name])

## Input
pseudobulk_input = pd.DataFrame(scgpt_input_pca, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs['scgpt_40PCs'] = pseudobulk_input

/tmp/ipykernel_1031071/1955282872.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(scgpt_input_pca, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### PCA

In [26]:
emb_name = 'X_pca'

## Input
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1031071/49313768.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### cell type props

In [27]:
## Input
celltype_counts_df = pd.DataFrame(adata.obs.groupby(sample_col).cell_type.value_counts()).rename({'count':'cell_type_counts'},axis=1).reset_index().pivot(index=sample_col, columns="cell_type", values="cell_type_counts")
celltype_fracs_df = celltype_counts_df.div(celltype_counts_df.sum(axis=1), axis=0)
pseudobulk_input = celltype_fracs_df
pbulk_embs['ct_props'] = pseudobulk_input

/tmp/ipykernel_1031071/702871507.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  celltype_counts_df = pd.DataFrame(adata.obs.groupby(sample_col).cell_type.value_counts()).rename({'count':'cell_type_counts'},axis=1).reset_index().pivot(index=sample_col, columns="cell_type", values="cell_type_counts")


#### pseudobulk of lognorm counts for HVGs

In [33]:
emb_name = "hvg_lognorm"

## Input
pseudobulk_input = pd.DataFrame(adata.layers['lognorm'][:,adata.var.highly_variable], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1031071/2059860499.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.layers['lognorm'][:,adata.var.highly_variable], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### pseudobulk of scaled counts for HVGs

In [34]:
emb_name = "hvg_scaled"

## Input
pseudobulk_input = pd.DataFrame(adata.X[:,adata.var.highly_variable], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1031071/1034296985.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.X[:,adata.var.highly_variable], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


### limit/order pseudobulk embeddings by sample_pool

In [40]:
this_pbulk_embs = {}
for emb in pbulk_embs:
    this_pbulk_embs[emb] = pbulk_embs[emb].loc[sample_pool]

## calculate heuristic baseline performance on these splits

not including majority class per study as a baseline, since we shouldn't have study based batch effects affecting performance using our GroupKFold!

In [41]:
val_majclass_bline_accs= []

for (train_index, val_index) in splits:
    #calculate baseline performance - majority class
    majclass_bline_pred = targets.iloc[train_index].value_counts().index[targets.iloc[train_index].value_counts().argmax()]
    if majclass_bline_pred not in targets.iloc[val_index].value_counts().index:
        val_majclass_bline_accs.append(0)
    else:
        val_majclass_bline_accs.append(targets.iloc[val_index].value_counts()[majclass_bline_pred]/len(val_index))

print("baseline accuracy on K val sets: ", np.around(val_majclass_bline_accs,3))
print("mean accuracy: ", np.round(np.mean(val_majclass_bline_accs),3))

baseline accuracy on K val sets:  [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
mean accuracy:  0.5


## Evaluate models using nested CV

In [42]:
## train KNN w/ Kfold CV
from sklearn.model_selection import GridSearchCV, cross_val_score

def nested_kfold_eval(pseudobulk_input, targets, splits, model, param_grid):

    # Inner CV for hyperparameter tuning
    inner_cv = GridSearchCV(model, param_grid, cv=5)

    # Nested CV
    nested_score = cross_val_score(inner_cv, pseudobulk_input, targets, cv=skf)
    
    print("Nested CV Score: ", np.round(nested_score.mean(),3))
    
    return nested_score

In [43]:
# set up a dict with models and hparams to test

model_param_dict = {
    'KNN': (KNeighborsClassifier(), {'n_neighbors':np.arange(10)+1}),
    'random_forest' : (RandomForestClassifier(), {'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}),
    'logistic_regression': (LogisticRegression(max_iter=500), {'C':[1e-5,1e-4,1e-3,1e-2,1e-1,1,10,100,1000,10000]})
}


In [44]:
task1_results = {}
for model in model_param_dict:
    print("--------- {} ----------".format(model))
    task1_results[model] = {}
    for emb in this_pbulk_embs: #our proxies arent meaningful when there is only one cell type
        print('embedding: ', emb)
        print('shape of embedding: ', this_pbulk_embs[emb].shape)
        task1_results[model][emb] = nested_kfold_eval(this_pbulk_embs[emb], targets, splits, model=model_param_dict[model][0], param_grid = model_param_dict[model][1])
        print("\n")
    print("\n")

--------- KNN ----------
embedding:  X_scGPT
shape of embedding:  (1000, 512)
Nested CV Score:  0.566


embedding:  scgpt_40PCs
shape of embedding:  (1000, 40)
Nested CV Score:  0.57


embedding:  X_pca
shape of embedding:  (1000, 50)
Nested CV Score:  0.674


embedding:  ct_props
shape of embedding:  (1000, 6)
Nested CV Score:  0.547


embedding:  hvg_scaled
shape of embedding:  (1000, 1523)
Nested CV Score:  0.675


embedding:  hvg_lognorm
shape of embedding:  (1000, 1523)
Nested CV Score:  0.63




--------- random_forest ----------
embedding:  X_scGPT
shape of embedding:  (1000, 512)
Nested CV Score:  0.628


embedding:  scgpt_40PCs
shape of embedding:  (1000, 40)
Nested CV Score:  0.62


embedding:  X_pca
shape of embedding:  (1000, 50)
Nested CV Score:  0.844


embedding:  ct_props
shape of embedding:  (1000, 6)
Nested CV Score:  0.533


embedding:  hvg_scaled
shape of embedding:  (1000, 1523)
Nested CV Score:  0.767


embedding:  hvg_lognorm
shape of embedding:  (1000, 1523)
Nes

In [ ]:
# means per cell type should perform the best
# let's do it with PCA for now which performed well above



#### mean embedding per cell type based on PCA -- acts as ORACLE

In [47]:
#get mean embedding per cell type

emb_name = 'X_pca'

# get mean embedding per sample+cell_type -- had to do some reindexing for this dataset specifically
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name]).merge(adata.obs[[sample_col,'cell_type']].reset_index().drop(columns=["index"]), right_index=True, left_index=True).groupby([sample_col,'cell_type']).mean()

# change columns to more interpretable strs 
pseudobulk_input.columns = ['PC'+str(c) for c in pseudobulk_input.columns]

# cell types that dont exist in a given sample - fill 0 for mean expression
pseudobulk_input = pseudobulk_input.fillna(0).reset_index()

# Pivot the dataframe
psblk_per_celltype = pseudobulk_input.pivot(index=sample_col, columns='cell_type')

# Flatten the multi-level columns
psblk_per_celltype.columns = ['_'.join(col).strip() for col in psblk_per_celltype.columns.values]

/tmp/ipykernel_1031071/3249380816.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name]).merge(adata.obs[[sample_col,'cell_type']].reset_index().drop(columns=["index"]), right_index=True, left_index=True).groupby([sample_col,'cell_type']).mean()


In [49]:
# concat cell type fracs and mean embs dfs
pbulk_embs['mean_pc_per_celltype'] = psblk_per_celltype

In [51]:
this_pbulk_embs = {}
for emb in pbulk_embs:
    this_pbulk_embs[emb] = pbulk_embs[emb].loc[sample_pool]

In [52]:
task1_results = {}
for model in model_param_dict:
    print("--------- {} ----------".format(model))
    task1_results[model] = {}
    for emb in ['mean_pc_per_celltype']: 
        print('embedding: ', emb)
        print('shape of embedding: ', this_pbulk_embs[emb].shape)
        task1_results[model][emb] = nested_kfold_eval(this_pbulk_embs[emb], targets, splits, model=model_param_dict[model][0], param_grid = model_param_dict[model][1])
        print("\n")
    print("\n")

--------- KNN ----------
embedding:  mean_pc_per_celltype
shape of embedding:  (1000, 300)
Nested CV Score:  0.99




--------- random_forest ----------
embedding:  mean_pc_per_celltype
shape of embedding:  (1000, 300)
Nested CV Score:  0.979




--------- logistic_regression ----------
embedding:  mean_pc_per_celltype
shape of embedding:  (1000, 300)
Nested CV Score:  0.989






In [ ]:
# stronger baseline - run Differential Expression on sample level between groups, pick top DEG and use those as input
# or actually just input the cancer genes that I know are the perturbed ones - oracle... tests if the mean across allcells is enough...
# but also want to run mean per cell type, which should be enough

# data: cd8 t cell (rarest cell population) pheno shift - 10,000 patients, subsetted to 5000 HVG

#### data is created in scSet/datasets/create_sim_data_scvi_pheno.ipynb (or a corresponding script)

In [6]:
h5ad_loc="/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_10000_patients_HVGonly_scGPT.h5ad"

In [7]:
# read in data from cache
adata = sc.read_h5ad(h5ad_loc)

/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
adata

AnnData object with n_obs × n_vars = 4450012 × 4451
    obs: 'orig_cellbarcode', 'patient_orig', 'cell_type', 'Tcellsubtype', 'group', 'patient'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_rank', 'variances', 'variances_norm', 'mean', 'std', 'index', 'id_in_vocab'
    uns: 'celltype_colors', 'hvg', 'log1p', 'pca'
    obsm: 'X_pca', 'X_scGPT'
    varm: 'PCs'
    layers: 'counts', 'lognorm'

In [9]:
adata.obs.rename(columns={'celltype':'cell_type'}, inplace=True)

### embed cells using scGPT

In [10]:
model_dir = Path("/data/rna_rep_learning/scGPT/scGPT_human")

In [11]:
adata = scg.tasks.embed_data(
    adata,
    model_dir,
    gene_col="index",
    batch_size=64,
    return_new_adata=False,
)

scGPT - INFO - match 4451/5000 genes in vocabulary of size 60697.


Embedding cells: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 69532/69532 [1:19:13<00:00, 14.63it/s]
/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/scgpt/tasks/cell_emb.py:279: ImplicitModificationWarning: Setting element `.obsm['X_scGPT']` of view, initializing view as actual.
  adata.obsm["X_scGPT"] = cell_embeddings
/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [12]:
#save this new adata
adata.write("/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_10000_patients_HVGonly_scGPT.h5ad", compression="gzip")


In [ ]:
adata = sc.read_h5ad("/data/rna_rep_learning/scset/synthetic_pheno_data/pheno_cd8t_fc4_10000_patients_HVGonly_scGPT.h5ad")

## define sample pool, target, X, splits for this task

### define sample_pool for this target

In [10]:
sample_col = 'patient'
target_col = 'group'

In [11]:
adata.obs[[sample_col, target_col]].drop_duplicates()[target_col].value_counts()

group
perturbed      5000
unperturbed    5000
Name: count, dtype: int64

In [12]:
sample_pool = adata.obs[sample_col].drop_duplicates()
print(len(sample_pool))

10000


In [13]:
sample_pool

0        sim_pt1
0        sim_pt2
0        sim_pt3
0        sim_pt4
0        sim_pt5
        ...     
0     sim_pt9996
0     sim_pt9997
0     sim_pt9998
0     sim_pt9999
0    sim_pt10000
Name: patient, Length: 10000, dtype: category
Categories (10000, object): ['sim_pt1', 'sim_pt2', 'sim_pt3', 'sim_pt4', ..., 'sim_pt9997', 'sim_pt9998', 'sim_pt9999', 'sim_pt10000']

### define target, and limit/order by sample_pool

In [14]:
## Define targets
targets = adata.obs.loc[:, [sample_col, target_col]].drop_duplicates().set_index(sample_col).loc[sample_pool,target_col]
labels, targets_ind = np.unique(targets, return_inverse=True)
targets = pd.Series(targets_ind, index=targets.index)

In [15]:
labels

array(['perturbed', 'unperturbed'], dtype=object)

In [16]:
targets.value_counts()/len(targets)

1    0.5
0    0.5
Name: count, dtype: float64

### 10 fold nested cross validation (cv outer loop)

In [17]:
skf = StratifiedKFold(n_splits=10)
splits = list(skf.split(np.zeros(len(targets)), targets))

In [18]:
for (train_index, test_index) in splits:
    #print("datasets reserved for test fold: ", list(adata.obs.set_index(sample_col).loc[sample_pool[test_index]].dataset.drop_duplicates()))
    print("n samples in train: ", len(train_index))
    print("n samples in test: ", len(test_index))
    print("\n")

n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000


n samples in train:  9000
n samples in test:  1000




## create pseudobulk embeddings to use downstream

In [19]:
pbulk_embs = {}

#### scGPT

In [20]:
emb_name = 'X_scGPT'

## Input
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1606150/1669887707.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### scGPT -> 40 PCs

In [21]:
emb_name = 'X_scGPT'

pca = sklearn.decomposition.PCA(n_components=40)
scgpt_input_pca = pca.fit_transform(adata.obsm[emb_name])

## Input
pseudobulk_input = pd.DataFrame(scgpt_input_pca, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs['scgpt_40PCs'] = pseudobulk_input

/tmp/ipykernel_1606150/1955282872.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(scgpt_input_pca, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### PCA

In [22]:
emb_name = 'X_pca'

## Input
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1606150/49313768.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name], index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### cell type props

In [23]:
## Input
celltype_counts_df = pd.DataFrame(adata.obs.groupby(sample_col).cell_type.value_counts()).rename({'count':'cell_type_counts'},axis=1).reset_index().pivot(index=sample_col, columns="cell_type", values="cell_type_counts")
celltype_fracs_df = celltype_counts_df.div(celltype_counts_df.sum(axis=1), axis=0)
pseudobulk_input = celltype_fracs_df
pbulk_embs['ct_props'] = pseudobulk_input

/tmp/ipykernel_1606150/702871507.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  celltype_counts_df = pd.DataFrame(adata.obs.groupby(sample_col).cell_type.value_counts()).rename({'count':'cell_type_counts'},axis=1).reset_index().pivot(index=sample_col, columns="cell_type", values="cell_type_counts")


#### pseudobulk of lognorm counts for HVGs

In [25]:
emb_name = "hvg_lognorm"

## Input
pseudobulk_input = pd.DataFrame(adata.layers['lognorm'][:,adata.var.highly_variable].todense(), index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1606150/1693436195.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.layers['lognorm'][:,adata.var.highly_variable].todense(), index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### pseudobulk of scaled counts for HVGs

In [28]:
emb_name = "hvg_scaled"

## Input
pseudobulk_input = pd.DataFrame(adata.X, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()

pbulk_embs[emb_name] = pseudobulk_input

/tmp/ipykernel_1606150/3949327578.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.X, index=adata.obs.loc[:,sample_col]).reset_index().groupby(sample_col).mean()


#### mean embedding per cell type based on PCA -- acts as ORACLE

In [29]:
#get mean embedding per cell type

emb_name = 'X_pca'

# get mean embedding per sample+cell_type -- had to do some reindexing for this dataset specifically
pseudobulk_input = pd.DataFrame(adata.obsm[emb_name]).merge(adata.obs[[sample_col,'cell_type']].reset_index().drop(columns=["index"]), right_index=True, left_index=True).groupby([sample_col,'cell_type']).mean()

# change columns to more interpretable strs 
pseudobulk_input.columns = ['PC'+str(c) for c in pseudobulk_input.columns]

# cell types that dont exist in a given sample - fill 0 for mean expression
pseudobulk_input = pseudobulk_input.fillna(0).reset_index()

# Pivot the dataframe
psblk_per_celltype = pseudobulk_input.pivot(index=sample_col, columns='cell_type')

# Flatten the multi-level columns
psblk_per_celltype.columns = ['_'.join(col).strip() for col in psblk_per_celltype.columns.values]

# concat cell type fracs and mean embs dfs
pbulk_embs['mean_pc_per_celltype'] = psblk_per_celltype

/tmp/ipykernel_1606150/1717705343.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pseudobulk_input = pd.DataFrame(adata.obsm[emb_name]).merge(adata.obs[[sample_col,'cell_type']].reset_index().drop(columns=["index"]), right_index=True, left_index=True).groupby([sample_col,'cell_type']).mean()


### delete adata to save memory

In [30]:
del adata

In [32]:
import gc
gc.collect()

543

### limit/order pseudobulk embeddings by sample_pool

In [31]:
this_pbulk_embs = {}
for emb in pbulk_embs:
    this_pbulk_embs[emb] = pbulk_embs[emb].loc[sample_pool]

## calculate heuristic baseline performance on these splits

not including majority class per study as a baseline, since we shouldn't have study based batch effects affecting performance using our GroupKFold!

In [33]:
val_majclass_bline_accs= []

for (train_index, val_index) in splits:
    #calculate baseline performance - majority class
    majclass_bline_pred = targets.iloc[train_index].value_counts().index[targets.iloc[train_index].value_counts().argmax()]
    if majclass_bline_pred not in targets.iloc[val_index].value_counts().index:
        val_majclass_bline_accs.append(0)
    else:
        val_majclass_bline_accs.append(targets.iloc[val_index].value_counts()[majclass_bline_pred]/len(val_index))

print("baseline accuracy on K val sets: ", np.around(val_majclass_bline_accs,3))
print("mean accuracy: ", np.round(np.mean(val_majclass_bline_accs),3))

baseline accuracy on K val sets:  [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
mean accuracy:  0.5


## Evaluate models using nested CV

In [34]:
## train KNN w/ Kfold CV
from sklearn.model_selection import GridSearchCV, cross_val_score

def nested_kfold_eval(pseudobulk_input, targets, splits, model, param_grid):

    # Inner CV for hyperparameter tuning
    inner_cv = GridSearchCV(model, param_grid, cv=5)

    # Nested CV
    nested_score = cross_val_score(inner_cv, pseudobulk_input, targets, cv=skf)
    
    print("Nested CV Score: ", np.round(nested_score.mean(),3))
    
    return nested_score

In [35]:
# set up a dict with models and hparams to test

model_param_dict = {
    'KNN': (KNeighborsClassifier(), {'n_neighbors':np.arange(10)+1}),
    'random_forest' : (RandomForestClassifier(), {'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}),
    'logistic_regression': (LogisticRegression(max_iter=500), {'C':[1e-5,1e-4,1e-3,1e-2,1e-1,1,10,100,1000,10000]})
}


In [36]:
task1_results = {}
for model in model_param_dict:
    print("--------- {} ----------".format(model))
    task1_results[model] = {}
    for emb in this_pbulk_embs: #our proxies arent meaningful when there is only one cell type
        print('embedding: ', emb)
        print('shape of embedding: ', this_pbulk_embs[emb].shape)
        task1_results[model][emb] = nested_kfold_eval(this_pbulk_embs[emb], targets, splits, model=model_param_dict[model][0], param_grid = model_param_dict[model][1])
        print("\n")
    print("\n")

--------- KNN ----------
embedding:  X_scGPT
shape of embedding:  (10000, 512)
Nested CV Score:  0.659


embedding:  scgpt_40PCs
shape of embedding:  (10000, 40)
Nested CV Score:  0.659


embedding:  X_pca
shape of embedding:  (10000, 50)
Nested CV Score:  0.776


embedding:  ct_props
shape of embedding:  (10000, 6)
Nested CV Score:  0.497


embedding:  hvg_lognorm
shape of embedding:  (10000, 4451)
Nested CV Score:  0.752


embedding:  hvg_scaled
shape of embedding:  (10000, 4451)
Nested CV Score:  0.738


embedding:  mean_pc_per_celltype
shape of embedding:  (10000, 300)
Nested CV Score:  0.993




--------- random_forest ----------
embedding:  X_scGPT
shape of embedding:  (10000, 512)
Nested CV Score:  0.708


embedding:  scgpt_40PCs
shape of embedding:  (10000, 40)
Nested CV Score:  0.698


embedding:  X_pca
shape of embedding:  (10000, 50)
Nested CV Score:  0.882


embedding:  ct_props
shape of embedding:  (10000, 6)
Nested CV Score:  0.495


embedding:  hvg_lognorm
shape of embed

/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/rpeyser/envs/scgpt_4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

Nested CV Score:  0.892


embedding:  hvg_scaled
shape of embedding:  (10000, 4451)
Nested CV Score:  0.887


embedding:  mean_pc_per_celltype
shape of embedding:  (10000, 300)
Nested CV Score:  0.994






In [39]:
for model in model_param_dict:
    print("--------- {} ----------".format(model))
    for emb in this_pbulk_embs: #our proxies arent meaningful when there is only one cell type
        print('embedding: ', emb)
        print('shape of embedding: ', this_pbulk_embs[emb].shape)
        print(np.round(task1_results[model][emb].mean(),3))
        print("\n")
    print("\n")

--------- KNN ----------
embedding:  X_scGPT
shape of embedding:  (10000, 512)
0.659


embedding:  scgpt_40PCs
shape of embedding:  (10000, 40)
0.659


embedding:  X_pca
shape of embedding:  (10000, 50)
0.776


embedding:  ct_props
shape of embedding:  (10000, 6)
0.497


embedding:  hvg_lognorm
shape of embedding:  (10000, 4451)
0.752


embedding:  hvg_scaled
shape of embedding:  (10000, 4451)
0.738


embedding:  mean_pc_per_celltype
shape of embedding:  (10000, 300)
0.993




--------- random_forest ----------
embedding:  X_scGPT
shape of embedding:  (10000, 512)
0.708


embedding:  scgpt_40PCs
shape of embedding:  (10000, 40)
0.698


embedding:  X_pca
shape of embedding:  (10000, 50)
0.882


embedding:  ct_props
shape of embedding:  (10000, 6)
0.495


embedding:  hvg_lognorm
shape of embedding:  (10000, 4451)
0.8


embedding:  hvg_scaled
shape of embedding:  (10000, 4451)
0.798


embedding:  mean_pc_per_celltype
shape of embedding:  (10000, 300)
0.995




--------- logistic_regressio

In [ ]:
# stronger baseline - run Differential Expression on sample level between groups, pick top DEG and use those as input
# or actually just input the cancer genes that I know are the perturbed ones - oracle... tests if the mean across allcells is enough...
# but also want to run mean per cell type, which should be enough